<a href="https://colab.research.google.com/github/cs432-websci-spr20/assignments/blob/master/Week_03_RTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 432/532 R Tutorial

We'll be doing some of the exercises from [R for Data Science](https://r4ds.had.co.nz) to get an introduction to R. We'll try doing this in Google Colab, but the commands can also be run locally using RStudio.

There are a ton of references available for R and since it's popular, you can pretty much search for whatever you want and find something close.

There are some examples of more of the statistical functions at
https://www.cs.odu.edu/~mweigle/courses/cs795/mklein-IntroR/lecture/

## First Thing

Install and load the tidyverse package.

In [0]:
install.packages("tidyverse")

In [0]:
library(tidyverse)

*Repeat the HW0 steps from [Section 3.2 First steps](https://r4ds.had.co.nz/data-visualisation.html#first-steps)*

Note that for any of these charts, choose 'Open Image in New Tab' for a larger version that you can save as a PNG.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy))

**HW0, Q8:** *Create a scatterplot of hwy vs. cyl.*

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = cyl))

## Basic Data Visualization in R

### Aesthetic mappings

*After repeating the HW0 setup, we'll pick up in [Section 3.3, Aesthetic mappings](https://r4ds.had.co.nz/data-visualisation.html#aesthetic-mappings)*

First, let's get a reminder of what's in the `mpg` dataset.  This command will pop up a help window with a description of the dataset.

In [0]:
?ggplot2::mpg

Here we'll print out what's in the `mpg` dataset.  Note that this is a "tibble" instead of a regular R "dataframe".  

If you're not familiar with either of these terms, don't worry about it.  If you are familiar with dataframes, then here's a description of the differences: https://r4ds.had.co.nz/tibbles.html#tibbles-vs.data.frame

In [0]:
mpg

Map the colors of datapoints to the `class` variable, indicating the class of each vehicle.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy, color = class))

Make all of the points blue.  Notice the difference in the placement of the color setting. It applies to all the dots and is not based on any data item.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy), color = "blue")

*This will generate an error.  Why?*

In [0]:
ggplot(data = mpg) 
+ geom_point(mapping = aes(x = displ, y = hwy))

### Facets
*Moving to [Section 3.5 Facets](https://r4ds.had.co.nz/data-visualisation.html#facets)*

We can create "small multiples" to show the data in separate charts. Note that both the x-axis range and the y-axis range is the same in all of the charts.

The only thing we've added here is the `facet_wrap` function.  It says to divide the charts by `class` and use 2 rows to display them.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy)) + 
  facet_wrap(~ class, nrow = 2)

### Statistical transformations

[Section 3.7](https://r4ds.had.co.nz/data-visualisation.html#statistical-transformations)

Time to look at bar charts and histograms.  We'll use a different dataset, describing diamonds.

In [0]:
?ggplot2::diamonds

The chart below uses `geom_bar` to generate a count of items with each type of cut.

In [0]:
ggplot(data = diamonds) + 
  geom_bar(mapping = aes(x = cut))

We can also use the `stat_count` function to generate the same chart.

In [0]:
ggplot(data = diamonds) + 
  stat_count(mapping = aes(x = cut))

We can use `stat_summary` to generate other summary statistics about the dataset. This is setting the min value of the line to the min value of `depth`, max to the max value, and the dot to the median value.


In [0]:
ggplot(data = diamonds) + 
  stat_summary(
    mapping = aes(x = cut, y = depth),
    fun.ymin = min,
    fun.ymax = max,
    fun.y = median
  )

*For more information on the stat functions available, see the [ggplot2 cheatsheet](https://github.com/rstudio/cheatsheets/blob/master/data-visualization-2.1.pdf).*

## Workflow: basics

[Section 4](https://r4ds.had.co.nz/workflow-basics.html)

*Now that we've made some charts, let go back to basics.*

Using R as a calculator:

In [0]:
1 / 200 * 30
(59 + 73 + 2) / 3
sin(pi/2)

**Important:** Variable assignment is done with `<-`, not with `=`

In [0]:
x <- 3*4
x

## Working with Data

[Section 5](https://r4ds.had.co.nz/transform.html) - uses the `flights` dataset and introduces filter(), arrange(), select()

In [0]:
install.packages("nycflights13")
library(nycflights13)
?nycflights13::flights

`filter()`  allows you to subset observations based on their values. 

In [0]:
filter(flights, month==1, day==1)

`arrange()` lets you sort rows (rather than filtering them out, just rearrange them.

In [0]:
arrange(flights, year, month, day)

`select()` lets you pick only certain columns

In [0]:
select(flights, year, month, day, tailnum)

Finally, we'll use `summarize()` and `group_by()` to perform summary operations on selected data.

This will compute the average departure delay by month.  (`na.rm = TRUE` just means to ignore any rows that have `NA` values)

In [0]:
by_month = group_by(flights, year, month)
summarize(by_month, delay = mean(dep_delay, na.rm = TRUE))

Here are a couple shortcut notations:
* `$` - allows you to reference a particular column (without having to use `filter()`)
* `%>%` - like a pipe (`|`) in unix

Here we're going to compute the average delay for all Delta flights.

In [0]:
delta = filter(flights, carrier=="DL")
mean(delta$dep_delay, na.rm=TRUE)

The pipe is just a shortcut.  This is the same result as above to compute the average departure delay by month.

You can omit the first parameter (dataset) and it's assumed that the data is coming from the pipe input.

In [0]:
group_by(flights, year, month) %>% summarize(delay = mean(dep_delay, na.rm = TRUE))

## Data Import

[Section 11](https://r4ds.had.co.nz/data-import.html) - getting data into R, reading CSV

The main function we'll look at is `read_csv()` to read in comma-separated files, but there are several others described in this section.

We can't mount Google Drive in Colab with R, but we can read CSVs that are online.

In [0]:
stars = read_csv("https://raw.githubusercontent.com/cs625-datavis-fall19/assignments/master/stars.csv")

Once this is read in, you can use all of the other functions that we've covered.

In [0]:
filter(stars, temp > 5000)

## Chart Labels

[Section 28](https://r4ds.had.co.nz/graphics-for-communication.html#label) Graphics for Communication

It's important to have good chart titles and labels.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy, color = class))

Let's add a title and caption.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy, color = class)) +
  labs(
    title="Fuel efficiency decreases with engine size",
    caption = "Data from fueleconomy.gov"
  )

Now let's change the axis labels.

In [0]:
ggplot(data = mpg) + 
  geom_point(mapping = aes(x = displ, y = hwy, color = class)) +
  labs(
    title="Fuel efficiency decreases with engine size",
    caption = "Data from fueleconomy.gov",
    x = "Engine displacement (L)",
    y = "Highway fuel economy (mpg)",
    color = "Car type"
  )